In [1]:
####################################################### KOA  ##################################################################
import glob
import os
import pandas as pd
import numpy as np
import calendar
from contextlib import contextmanager
from datetime import datetime
import math
import warnings
warnings.filterwarnings("ignore")

In [2]:
os.chdir(r'C:\Users\becho\Documents\MasterReport\Reporting\Bases Manuales\11 FILL RATE\KOA')
FL = []
csv_files = glob.glob('*.xlsx')

In [3]:
for filename in csv_files:
    data = pd.read_excel(filename,sheet_name='Sheet1')
    data = data.drop([0,1],axis=0)
    FL.append(data)

In [4]:
FL = pd.concat(FL,ignore_index=True)
FL.rename(columns = {'Unnamed: 2': 'COD_CLI','Unnamed: 8':'SKU','Unnamed: 9':'PEDIDO','Unnamed: 3':'Fecha_t','Unnamed: 10':'DIF_PRECIO','Unnamed: 11':'OT_DIFERENCIA','Unnamed: 12':'QUIEBRE','Unnamed: 13':'CAPACIDAD','Unnamed: 14':'RECHAZO','Unnamed: 15':'SIN_LIQUIDAR'} ,inplace=True)
FL=FL[["Fecha_t","COD_CLI","SKU","PEDIDO","DIF_PRECIO","OT_DIFERENCIA","QUIEBRE","CAPACIDAD","RECHAZO","SIN_LIQUIDAR"]]

In [5]:
FL.PEDIDO = FL.PEDIDO.astype(float)
FL.PEDIDO = FL.PEDIDO.round(1)
FL.DIF_PRECIO = FL.DIF_PRECIO.astype(float)
FL.DIF_PRECIO = FL.DIF_PRECIO.round(1)
FL.OT_DIFERENCIA = FL.OT_DIFERENCIA.astype(float)
FL.OT_DIFERENCIA = FL.OT_DIFERENCIA.round(1)
FL.QUIEBRE = FL.QUIEBRE.astype(float)
FL.QUIEBRE = FL.QUIEBRE.round(1)
FL.CAPACIDAD = FL.CAPACIDAD.astype(float)
FL.CAPACIDAD = FL.CAPACIDAD.round(1)
FL.RECHAZO = FL.RECHAZO.astype(float)
FL.RECHAZO = FL.RECHAZO.round(1)
FL.SIN_LIQUIDAR = FL.SIN_LIQUIDAR.astype(float)
FL.SIN_LIQUIDAR = FL.SIN_LIQUIDAR.round(1)

In [6]:
FL_P=FL[["Fecha_t","COD_CLI","SKU","PEDIDO"]]
FL_P.rename(columns = {'PEDIDO':'VALOR'},inplace=True)
FL_P['Categoria']="Solicitado"

FL_DP=FL[["Fecha_t","COD_CLI","SKU","DIF_PRECIO"]]
FL_DP.rename(columns = {'DIF_PRECIO':'VALOR'},inplace=True)
FL_DP['Categoria']="Diferencia_de_precio"

FL_OT=FL[["Fecha_t","COD_CLI","SKU","OT_DIFERENCIA"]]
FL_OT.rename(columns = {'OT_DIFERENCIA':'VALOR'},inplace=True)
FL_OT['Categoria']="OT_DIFERENCIAS"

FL_Q=FL[["Fecha_t","COD_CLI","SKU","QUIEBRE"]]
FL_Q.rename(columns = {'QUIEBRE':'VALOR'},inplace=True)
FL_Q['Categoria']="Quiebre"

FL_C=FL[["Fecha_t","COD_CLI","SKU","CAPACIDAD"]]
FL_C.rename(columns = {'CAPACIDAD':'VALOR'},inplace=True)
FL_C['Categoria']="Capacidad"

FL_R=FL[["Fecha_t","COD_CLI","SKU","RECHAZO"]]
FL_R.rename(columns = {'RECHAZO':'VALOR'},inplace=True)
FL_R['Categoria']="Rechazo"

FL_SL=FL[["Fecha_t","COD_CLI","SKU","SIN_LIQUIDAR"]]
FL_SL.rename(columns = {'SIN_LIQUIDAR':'VALOR'},inplace=True)
FL_SL['Categoria']="Sin_liquidar"

In [7]:
FL_KOA = FL_DP.append(FL_P)
FL_KOA = FL_KOA.append(FL_OT)
FL_KOA = FL_KOA.append(FL_Q)
FL_KOA = FL_KOA.append(FL_C)
FL_KOA = FL_KOA.append(FL_R)
FL_KOA = FL_KOA.append(FL_SL)

In [8]:
os.chdir(r'C:\Users\becho\Documents\MasterReport\Maestras')
Maestro_SKU = pd.read_excel('Materiales  Abi.xlsx', sheet_name='Hoja1')#, dtype ={'Cj/Pall simil':np.float64,'HL/cj':np.float64} ,decimal='.')
Maestro_SKU.rename(columns = {'Material': 'SKU_KOA','Material ABI': 'Sku','HL/cj':'HL_cj'} ,inplace=True)

FL_KOA.rename(columns = {'SKU': 'Sku'} ,inplace=True)

Maestra_SKU_KOA = Maestro_SKU[["SKU_KOA","Sku"]]

In [9]:
FL_KOA =FL_KOA.dropna(subset=['Fecha_t'])
FL_KOA.Fecha_t = FL.Fecha_t.astype(str)

Titulo =FL_KOA[["Fecha_t"]]
Titulo = Titulo["Fecha_t"].str.split("-", n=2, expand=True)

aa=Titulo[2].str.split(" ", n=1, expand=True) #dia dd
Titulo[0] = Titulo[0].astype(str)             #anho yyyy
Titulo[1] = Titulo[1].astype(str)             #mes  mm
aa[0] = aa[0].astype(str)                     #dia str dd

FL_KOA['Fecha_t'] = Titulo[0]+Titulo[1]+aa[0] #yyyymmdd

FL_KOA =FL_KOA[["Fecha_t","COD_CLI", "Sku", "Categoria","VALOR"]]

In [10]:
########################################   SI      #####################################
os.chdir(r'C:\Users\becho\Documents\MasterReport\Reporting\Bases Manuales\001 Sharepoint Jorge')
Sh_j = []
csv_files = glob.glob('*.txt')

for filename in csv_files:
    if filename == 'Noviembre.txt':
        data = pd.read_csv(filename,encoding='ANSI', sep='\t')#,dtype={'Stock_Cajas':np.float64, 'Sku_ABInBev': np.float64} ,decimal='.')
        Sh_j.append(data)
Sh_j = pd.concat(Sh_j,ignore_index=True)

In [11]:
Sh_j = Sh_j[Sh_j["cga_tip_cga"]=="LOGISTICA VFILLRATE_TRADICSUPERM_DIARIO"]
Sh_j = Sh_j.dropna(subset=['log_anno_mes_dia_ent'])
Sh_j = Sh_j[['log_anno_mes_dia_ent', 'log_cod_cliente','log_sku_producto','log_mot_no_ent','log_cjs_sol','log_can_cjs_rech']]

In [12]:
Sh_j.log_can_cjs_rech = Sh_j.log_can_cjs_rech.astype(float)
Sh_j.log_can_cjs_rech = Sh_j.log_can_cjs_rech.round(3)
Sh_j.log_cjs_sol = Sh_j.log_cjs_sol.astype(float)
Sh_j.log_cjs_sol = Sh_j.log_cjs_sol.round(3)
Sh_j.rename(columns = {'log_anno_mes_dia_ent':'Fecha_t','log_cod_cliente':'COD_CLI','log_sku_producto':'Sku_KOE','log_mot_no_ent':'ATRIBUTO','log_can_cjs_rech': 'VALOR'} ,inplace=True)

In [13]:
#BASE rechazos
FL_Sh = Sh_j
FL_Sh["ATRIBUTO"]=FL_Sh.ATRIBUTO.str.replace(r"MOTIVO_DIF_COSTO","Diferencia_de_precio")
FL_Sh["ATRIBUTO"]=FL_Sh.ATRIBUTO.str.replace(r"MOTIVO_ELIMINADAS","Eliminado")
FL_Sh["ATRIBUTO"]=FL_Sh.ATRIBUTO.str.replace(r"MOTIVO_ERROR_MAESTRA","ERROR_MAESTRA")
FL_Sh["ATRIBUTO"]=FL_Sh.ATRIBUTO.str.replace(r"MOTIVO_LIMITE_DE_CREDITO","Diferencia_de_precio")
FL_Sh["ATRIBUTO"]=FL_Sh.ATRIBUTO.str.replace(r"MOTIVO_QUIEBRE","Quiebre")
FL_Sh["ATRIBUTO"]=FL_Sh.ATRIBUTO.str.replace(r"MOTIVO_RETORNOS","Rechazo")
FL_Sh["ATRIBUTO"]=FL_Sh.ATRIBUTO.str.replace(r"MOTIVO_NO_IDENTIFICADAS","Otros")

In [14]:
FL_Sh.rename(columns = {'ATRIBUTO':'Categoria'} ,inplace=True)
FL_Sh =FL_Sh[["Fecha_t","COD_CLI", "Sku_KOE", "Categoria","VALOR"]]
FL_Sh = FL_Sh[FL_Sh["VALOR"]>0]
Sh_SOL = Sh_j
Sh_j = Sh_j[Sh_j["log_cjs_sol"]>0]

In [15]:
Sh_SOL.rename(columns = {'ATRIBUTO':'Categoria'} ,inplace=True)
Sh_SOL = Sh_SOL[(Sh_SOL["Categoria"]=="Eliminado")]
Sh_SOL.Categoria="Solicitado"
Sh_SOL =Sh_SOL[["Fecha_t","COD_CLI", "Sku_KOE", "Categoria","log_cjs_sol"]]
Sh_SOL.rename(columns = {'log_cjs_sol': 'VALOR'} ,inplace=True)
FL_Sh_c = FL_Sh.append(Sh_SOL)

In [32]:
#Sh_SOL[Sh_SOL.Fecha_t.isin([20211128,20211129,20211130])].VALOR.sum()

72716.0

In [34]:
#FL_Sh_c[FL_Sh_c.Fecha_t.isin([20211128,20211129,20211130])].VALOR.sum()

92532.0

In [17]:
os.chdir(r'C:\Users\becho\Documents\MasterReport\Reporting\Maestras')
Maestro_SKU = pd.read_excel('Materiales  Abi.xlsx', sheet_name='Hoja1')#, type ={'Cj/Pall simil':np.float64,'HL/cj':np.float64} ,decimal='.')
Maestro_SKU.rename(columns = {'SKU KOE': 'Sku_KOE','Material ABI': 'Sku','HL/cj':'HL_cj'} ,inplace=True)
Maestro_SKU.HL_cj = np.round(Maestro_SKU.HL_cj,5)
Maestro_SKU.HL_cj =Maestro_SKU.HL_cj/10
Maestro_SKU.HL_cj = np.round(Maestro_SKU.HL_cj,6)

In [18]:
#Maestro_SKU[["Sku_KOE","Sku"]].to_excel(r"C:\Users\becho\Documents\MasterReport\maestro_sku_koe.xls")

In [19]:
FL_Sh_c2 = FL_Sh_c
Maestra_SKU_KOA = Maestro_SKU[["Sku_KOE","Sku"]]
Maestra_SKU_KOA = Maestra_SKU_KOA.drop_duplicates(subset=["Sku_KOE"])
FL_Sh_c2 = pd.merge(FL_Sh_c2,Maestra_SKU_KOA,on=["Sku_KOE"],how="outer")

In [35]:
#FL_Sh_c2[FL_Sh_c2.Fecha_t.isin([20211128,20211129,20211130])].VALOR.sum()

92261.0

In [20]:
FL_Sh_c2.Sku = FL_Sh_c2.Sku.astype(str)
FL_Sh_c2.Sku = FL_Sh_c2.Sku.str.split(".", n=1, expand=True)[0]

In [21]:
FL_Sh_c2.COD_CLI = FL_Sh_c2.COD_CLI.astype(str)
FL_Sh_c2.COD_CLI = FL_Sh_c2.COD_CLI.str.split(".", n=1, expand=True)[0]

In [22]:
FL_Sh_c2 = FL_Sh_c2.dropna(subset=['Sku'])

In [36]:
#FL_Sh_c2[FL_Sh_c2.Fecha_t.isin([20211128,20211129,20211130])].VALOR.sum()

92261.0

In [23]:
FL_Sh_c2 = FL_Sh_c2.dropna(subset=['Fecha_t','COD_CLI','Sku_KOE','Categoria','VALOR'])
FL_Sh_c2.to_csv(r"C:\Users\becho\Documents\MasterReport\Reporting\ETL\07 Fill Rate\FL_merg.txt")
FL_Sh_c2['KO']="KOE"

FL_Sh_c2['val'] = FL_Sh_c2.VALOR.astype(str)
FL_Sh_c2['Id'] = FL_Sh_c2.Fecha_t.astype(str) +"-"+ FL_Sh_c2.COD_CLI +"-"+FL_Sh_c2.Sku.astype(str)+"-"+FL_Sh_c2.Categoria.astype(str)+"-"+FL_Sh_c2.val
FL_Sh_c2 = FL_Sh_c2.drop_duplicates(subset=["Id"])

In [37]:
#FL_Sh_c2[FL_Sh_c2.Fecha_t.isin([20211128,20211129,20211130])].VALOR.sum()

92261.0

In [24]:
##### Error Cenco ###########
FL_Sh_c3 = (FL_Sh_c2[
                 ((FL_Sh_c2["Fecha_t"]==20210628) | (FL_Sh_c2["Fecha_t"]==20210629)| (FL_Sh_c2["Fecha_t"]==20210630))
                  & (FL_Sh_c2["Categoria"]=="Eliminado")
                    ])
FL_Sh_c3.VALOR = FL_Sh_c3.VALOR*-1
FL_Sh_c2 = FL_Sh_c2.append(FL_Sh_c3)

In [25]:
################################### DATA FINAL ###########################################
FL = FL.append(FL_Sh_c2)

FL = FL[["Fecha_t", "COD_CLI", "Sku", "Categoria", "VALOR", "KO"]]
FL = FL.dropna(subset=['Fecha_t','Sku',"Categoria", "VALOR"])
FL = FL.drop(FL[FL['Sku']=='nan'].index)
FL = FL.drop(FL[FL['Categoria']=='nan'].index)
FL = FL.drop(FL[FL['Fecha_t']=='nan'].index)

FL.to_csv(r"C:\Users\becho\Documents\MasterReport\Reporting\ETL\07 Fill Rate\FILL_RATE.txt")

In [27]:
#FL[FL['KO']=='KOE']['Fecha_t'].astype(int).max()
FL[(FL.KO == 'KOE') & (FL.Fecha_t.isin([20211128,20211129,20211130]))].VALOR.sum()

85463.0

In [28]:
###################################### FIN #####################################